# trcFormat Converter
### Johanna Rahm, Sebastian Malkusch
#### Research group Heilemann
#### Institute for Physical and Theoretical Chemistry, Goethe University Frankfurt a.M.

## Load file

Insert the full file name & direcory path of the tracked file with .txt ending or browse for it. (Example path: F:\Marburg\single_colour_tracking\resting\160404_CS5_Cell1\cell_1_MMStack_Pos0.ome.tif.tracked.txt)

In [1]:
from pySPT.widgets import widgetDirectoryStructure
from pySPT.widgets import widgetTrcFormat
from pySPT.analysis import trcFormat
widget_dir_structure = widgetDirectoryStructure.WidgetDirStructure()
widget_trc_format = widgetTrcFormat.WidgetTrcFormat()
trc_format = trcFormat.TrcFormat()
display(widget_trc_format.file_text_box, widget_trc_format.file_button)
widget_trc_format.file_button.on_click(widget_trc_format.open_file)
widget_trc_format.file_text_box.observe(widget_trc_format.change_file_box)

Text(value='path', description='Complete path', placeholder='Type something', style=DescriptionStyle(descripti…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

## Run analysis

A new file with columns: seg_id, frame, x, y, intensity will be created.

In [2]:
display(widget_trc_format.run_button)
def run_analysis(event):
    widget_trc_format.clear_output
    if widget_trc_format.got_file_name:
        trc_format.file_name = widget_trc_format.file_text_box.value
        trc_format.run_trc_format()
    else:
        print("No file was loaded.")
widget_trc_format.run_button.on_click(run_analysis)


Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

[   6.   24.   20. ... 5889. 5854. 5888.]


## Save analysis

In [3]:
display(widget_trc_format.save_button)
def save_analysis(event):
    widget_trc_format.clear_output
    widget_dir_structure.name_handling(widget_trc_format.file_name)
    widget_dir_structure.create_raw_base_name()
    widget_dir_structure.create_folder()
    trc_format.save_trc_file(widget_dir_structure.pre_analysis_dir, widget_dir_structure.raw_base_name)
widget_trc_format.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')